In [2]:
#| default_exp coinbase

In [3]:
#|hide
%load_ext autoreload
%autoreload 2

## Coinbase

Manages the coinbase data download and processing. It is important to note that the Coinbase API might change from time to time, and the functions might need to be updated. You can find the documentation here: https://docs.cdp.coinbase.com/exchange/reference/exchangerestapi_getproductcandles.

Notice also that there are limits of what one can download. See the documentation for more details. In particular: https://docs.cdp.coinbase.com/exchange/docs/rest-rate-limits




In [151]:
#| export
from datetime import datetime
import pandas as pd
import requests
import os
import datetime as dt


In [171]:
#| export

def retrieve_coinbase_price(pair="BTC-USD",time_interval=3600, 
                            end_date=datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'), 
                            start_date=(datetime.now()-pd.Timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')):
    # This function retrieves the price data from Coinbase for a given pair and time interval.
    # There is a limit of 300 requests per hour.
    # There is a maximum of 200 candles per request. If you need more, you need to paginate.
    # If you try to retrive more per hour or more candles per request, you will get a error, and the function will return None.
    # General URL is: https://api.exchange.coinbase.com/products/{pair}/candles
    # Important: time_interval is in seconds
    # For documentation see: https://docs.cdp.coinbase.com/exchange/reference/exchangerestapi_getproductcandles
    
    url = f"https://api.exchange.coinbase.com/products/{pair}/candles?granularity={time_interval}&start={start_date}&end={end_date}"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    data = response.json()
    if not data:
        return None

    column_names = ['unix', 'low', 'high', 'open', 'close', 'volume']
    df = pd.DataFrame(data, columns=column_names)
    df['datetime'] = pd.to_datetime(df['unix'], unit='s')
    df['pair'] = pair
    df = df.sort_values(by='datetime')
    # drop unix column and put date first
    df = df.drop(columns=['unix'])
    df = df[['datetime'] + [col for col in df.columns if col != 'datetime']]
    return df


In [141]:
#| export

def coinbase_tokens():
    currencies_url = "https://api.exchange.coinbase.com/products"
    currencies_response = requests.get(currencies_url)
    currencies_data = currencies_response.json()
    return pd.DataFrame(currencies_data)


In [142]:
#| export
def coinbase_usd_tokens():
    tokens = coinbase_tokens()
    return tokens[tokens['quote_currency']=='USD']


In [170]:
#| export

# Download long time series from Coinbase accounting for maximum number of dates per request
def coinbase_price_history(pair='BTC-USD',start_date='2024-01-01',end_date='2024-05-01',time_interval=3600,max_pull=250,verbose=False):
    # This function downloads the price history for a given pair from Coinbase.
    # It accounts for the maximum number of dates that can be downloaded at once. This can change in the future.

    # convert the dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if end_date < start_date:
        # raise an error
        raise ValueError("End date is smaller than start date")
    # calculate the number of hours between the start and end date
    n_hours = (end_date - start_date).days * 24
    if n_hours <= max_pull:
        df = retrieve_coinbase_price(pair,start_date=start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),end_date=end_date.strftime('%Y-%m-%dT%H:%M:%SZ'),time_interval=time_interval)
        return df
    # create a timeline with the maximum number of dates per request
    time_line = pd.date_range(start=start_date, end=end_date, periods=n_hours//max_pull+1)
    price_list = []
    for i in range(len(time_line)-1):
        end = time_line[i+1]
        start = time_line[i]
        if verbose:
            print(f"Downloading data from {start} to {end}")
        df = retrieve_coinbase_price(pair,start_date=start.strftime('%Y-%m-%dT%H:%M:%SZ'),end_date=end.strftime('%Y-%m-%dT%H:%M:%SZ'),time_interval=time_interval)
        if df is not None:
            price_list.append(df)
    # concatenate the dataframes and sort by date and drop duplicates dates
    df = pd.concat(price_list)
    df = df.sort_values(by='datetime')
    df = df.drop_duplicates(subset='datetime').reset_index(drop=True)
    return df

In [162]:
#| export

def save_file(df,folder_path,file_name,type="csv"):
    if type == "csv":
        df.to_csv(f"{folder_path}/{file_name}.csv",index=False)
    elif type == "parquet":
        df.to_parquet(f"{folder_path}/{file_name}.parquet")



In [245]:
#| export

def coinbase_to_file(folder_path="../data/coinbase",token_list=coinbase_usd_tokens()['id'].tolist(),type="csv",
                     interval=3600,all_tokens=True):
    # Documentation:
    # This function downloads the price history for a given list of tokens from Coinbase and saves them in a folder.
    # If the folder does not exist, it creates it.
    # If the folder exists, it reads the existing files and appends the new data to them.
    # The files are saved in the folder with the name of the token and the extension of the type.
    # The files are saved in the format of the type.
    
    # create the folder if it does not exist, and if it exists, read the file names
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        if all_tokens:
            file_names = os.listdir(folder_path)
            # remove the file extension
            tokens_in_folder = [os.path.splitext(file_name)[0] for file_name in file_names]
            # join tokens in folder with token_list and remove the duplicates
            token_list = list(set(token_list + tokens_in_folder))
    # loop over the token list and open the file if it exists and append the new data
    for token in token_list:
        print(f"Processing {token}")
        file_name = f"{folder_path}/{token}.{type}"
        if os.path.exists(file_name):
            if type == "csv":
                df = pd.read_csv(file_name)
            elif type == "parquet":
                df = pd.read_parquet(file_name)
            else:
                raise ValueError(f"Type {type} not supported")
            # read last date in the file
            last_date = pd.to_datetime(df['datetime'].iloc[-1])
            if interval == 3600:
                today = datetime.now(tz=dt.UTC).replace(minute=0,second=0,microsecond=0)
            else:
                today = datetime.now(tz=dt.UTC)
            #print(last_date.tz_localize(dt.UTC),today)
            if last_date.tz_localize(dt.UTC) < today:
                df_new = coinbase_price_history(pair=token,
                                            start_date=last_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                            end_date=today.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                            time_interval=interval)
                df = pd.concat([df,df_new])
                # drop duplicates and sort by date
                df = df.drop_duplicates(subset='datetime').sort_values(by='datetime').reset_index(drop=True)
                save_file(df,folder_path,token,type)
            else:
                print(f"File {token} is up to date")
        else:
            df = coinbase_price_history(pair=token,
                                        start_date='2016-01-01T00:00:00Z',
                                        end_date=datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
                                        time_interval=interval)
            save_file(df,folder_path,token,type)
    


In [268]:
#| export
def read_all_files(folder_path="../data/coinbase",type="csv"):
    file_names = os.listdir(folder_path)
    df_list = []
    for file_name in file_names:
        if type == "csv":
            df = pd.read_csv(f"{folder_path}/{file_name}")
        elif type == "parquet":
            df = pd.read_parquet(f"{folder_path}/{file_name}")
        else:
            raise ValueError(f"Type {type} not supported")
        df_list.append(df)
    return pd.concat(df_list)



## Examples

List the available tokens on Coinbase.

In [83]:
#| eval: false
tokens = coinbase_tokens()
print(tokens)

                id base_currency quote_currency quote_increment  \
0          LTC-GBP           LTC            GBP            0.01   
1          UMA-EUR           UMA            EUR           0.001   
2         IOTX-EUR          IOTX            EUR         0.00001   
3          VGX-EUR           VGX            EUR          0.0001   
4      BIGTIME-USD       BIGTIME            USD         0.00001   
..             ...           ...            ...             ...   
641       ZEN-USDT           ZEN           USDT            0.01   
642      1INCH-BTC         1INCH            BTC       0.0000001   
643        JTO-USD           JTO            USD          0.0001   
644      NEST-USDT          NEST           USDT         0.00001   
645  CORECHAIN-USD     CORECHAIN            USD           0.001   

    base_increment   display_name min_market_funds  margin_enabled  post_only  \
0       0.00000001        LTC-GBP             0.72           False      False   
1            0.001        UMA-EUR

To make it simpler, we can filter the tokens by the quote currency. In this case, we filter for USD.


In [85]:
#| eval: false
usd_tokens = coinbase_usd_tokens()
print(usd_tokens)


             id base_currency quote_currency quote_increment base_increment  \
0    PIRATE-USD        PIRATE            USD          0.0001            0.1   
5       DOT-USD           DOT            USD           0.001     0.00000001   
6    AURORA-USD        AURORA            USD           0.001           0.01   
7       UPI-USD           UPI            USD         0.00001              1   
8      OMNI-USD          OMNI            USD           0.001           0.01   
..          ...           ...            ...             ...            ...   
640    RARI-USD          RARI            USD            0.01          0.001   
641   SUPER-USD         SUPER            USD         0.00001           0.01   
642    HIGH-USD          HIGH            USD           0.001           0.01   
644     TRB-USD           TRB            USD            0.01          0.001   
645     DNT-USD           DNT            USD          0.0001            0.1   

    display_name min_market_funds  margin_enabled  

Download the historical price of BTC-USD for few months at every hour. IMPORTANT: datetime is in UTC!


In [172]:
#| eval: false
df = coinbase_price_history()
print(df)

                datetime       low      high      open     close       volume  \
0    2024-01-01 00:00:00  42261.58  42543.64  42288.58  42452.66   379.197253   
1    2024-01-01 01:00:00  42415.00  42749.99  42453.83  42594.68   396.201924   
2    2024-01-01 02:00:00  42488.03  42625.68  42594.58  42571.32   227.141166   
3    2024-01-01 03:00:00  42235.00  42581.26  42571.32  42325.11   306.005694   
4    2024-01-01 04:00:00  42200.00  42393.48  42325.10  42389.77   296.233644   
...                  ...       ...       ...       ...       ...          ...   
2900 2024-04-30 20:00:00  59087.36  60082.82  59098.18  59869.99  1655.908771   
2901 2024-04-30 21:00:00  59862.17  60326.56  59865.66  60100.36   615.103455   
2902 2024-04-30 22:00:00  60070.00  60990.48  60100.01  60526.64   727.224395   
2903 2024-04-30 23:00:00  60330.92  60878.39  60529.65  60622.10   346.354179   
2904 2024-05-01 00:00:00  60015.99  60785.49  60621.20  60173.03   499.935743   

         pair  
0     BTC-U

Download all history available in coinbase for the tokens saved in folder "../data/coinbase" 
This process can take a while specially if you are downloading many tokens the first time.


In [248]:
#| eval: false
coinbase_to_file(token_list=['OP-USD'],type="parquet")

Processing OP-USD
File OP-USD is up to date
Processing AAVE-USD
File AAVE-USD is up to date


Now read the file from folder "../data/coinbase":


In [247]:
#| eval: false
df = pd.read_parquet("../data/coinbase/AAVE-USD.parquet")
print(df)

                 datetime      low     high     open    close    volume  \
0     2020-12-15 17:00:00   87.700   90.400   90.400   88.347  4289.548   
1     2020-12-15 18:00:00   85.620   88.407   88.010   86.000  3802.529   
2     2020-12-15 19:00:00   84.575   89.500   86.105   85.952  2835.124   
3     2020-12-15 20:00:00   84.828   85.952   85.952   85.599   798.670   
4     2020-12-15 21:00:00   84.744   85.893   85.599   85.208   442.464   
...                   ...      ...      ...      ...      ...       ...   
34263 2024-11-12 13:00:00  172.640  177.580  177.420  176.540  4893.095   
34264 2024-11-12 14:00:00  175.430  180.380  176.640  180.260  4112.704   
34265 2024-11-12 15:00:00  177.400  180.860  180.160  177.450  6045.649   
34266 2024-11-12 16:00:00  174.740  178.960  177.460  178.530  4194.807   
34267 2024-11-12 17:00:00  178.200  179.230  178.980  178.850   383.705   

           pair  
0      AAVE-USD  
1      AAVE-USD  
2      AAVE-USD  
3      AAVE-USD  
4      AA

Full download with no files in the folder takes almost 5 hours. After that, it is much faster (~ 6 minutes).


In [249]:
#| eval: false
coinbase_to_file(type="parquet") # takes almost 5 hours if there no files in the folder!

Processing ATOM-USD
Processing DYP-USD
Processing ASM-USD
Processing ENJ-USD
Processing RAD-USD
Processing COMP-USD
Processing ZK-USD
Processing ZETA-USD
Processing FET-USD
Processing RPL-USD
Processing OP-USD
File OP-USD is up to date
Processing HBAR-USD
Processing INJ-USD
Processing DDX-USD
Processing 1INCH-USD
Processing TRIBE-USD
Processing LRC-USD
Processing MLN-USD
Processing AAVE-USD
File AAVE-USD is up to date
Processing POL-USD
Processing GRT-USD
Processing MATIC-USD
Processing GST-USD
Processing WCFG-USD
Processing C98-USD
Processing BNT-USD
Processing MPL-USD
Processing BLAST-USD
Processing BLZ-USD
Processing PRIME-USD
Processing UMA-USD
Processing EGLD-USD
Processing DAI-USD
Processing MDT-USD
Processing RNDR-USD
Processing COVAL-USD
Processing OOKI-USD
Processing LIT-USD
Processing MSOL-USD
Processing MATH-USD
Processing RGT-USD
Processing MULTI-USD
Processing UNI-USD
Processing TONE-USD
Processing ANT-USD
Processing GHST-USD
Processing SUI-USD
Processing DREP-USD
Processi

Read all files in the folder into a single dataframe:


In [ ]:
#| eval: false
df = read_all_files(type="parquet")
print(df)